In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_selection import VarianceThreshold 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', None)
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE


In [3]:
num = pd.read_csv('numerical.csv')

In [4]:
cat = pd.read_csv('categorical.csv')
target = pd.read_csv('target.csv')

In [5]:
num.dtypes

TCODE         int64
AGE         float64
INCOME        int64
WEALTH1       int64
HIT           int64
             ...   
AVGGIFT     float64
CONTROLN      int64
HPHONE_D      int64
RFA_2F        int64
CLUSTER2      int64
Length: 315, dtype: object

In [6]:
cat.dtypes

STATE            object
CLUSTER           int64
HOMEOWNR         object
GENDER           object
DATASRCE          int64
RFA_2R           object
RFA_2A           object
GEOCODE2         object
DOMAIN_A         object
DOMAIN_B          int64
ODATEW_YR         int64
ODATEW_MM         int64
DOB_YR            int64
DOB_MM            int64
MINRDATE_YR       int64
MINRDATE_MM       int64
MAXRDATE_YR       int64
MAXRDATE_MM       int64
LASTDATE_YR       int64
LASTDATE_MM       int64
FIRSTDATE_YR      int64
FIRSTDATE_MM    float64
dtype: object

In [7]:
df = pd.concat((num,cat,target),axis=1)
df

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11.0,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10.0,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5

In [8]:
df.fillna(df.mean(), inplace=True)

/var/folders/2t/2pxpfxm91mg6w4grvjd9xpr00000gn/T/ipykernel_32028/820435583.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)


In [48]:
y = df['TARGET_B']
X = df.drop(['TARGET_B'], axis = 1)

# Random Forest

In [49]:
numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object) 

# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out()) # needed to avoid error
X = pd.concat([numericalX, encoded_categorical], axis = 1)

# Note: we need to do train/test split before downsampling, and then only downsample the training set - Why?
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [53]:
# for oversampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)
# quicker way to oversample category 1:
category_1 = trainset[trainset['TARGET_B']==1].sample(len(trainset[trainset['TARGET_B']==0]),
                                                      replace=True)


category_0 = trainset[trainset['TARGET_B']== 0 ]
print(category_0.shape)
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
# #data = data.reset_index(drop=True)
print(X_train.shape)

(72486, 356)
(144972, 355)


In [54]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6214993240073945
0.6114342608604517


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11128,  6955],
       [  460,   540]])

In [107]:
y_pred_df = pd.DataFrame({'y_pred': y_pred}, index=X_test.index)
y_pred_df

,y_pred
58053,1
9484,0
13395,0
1466,1
2076,0
...,...
94255,1
26449,0
1969,0
48574,1


In [57]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.62      0.75     18083
           1       0.07      0.54      0.13      1000

    accuracy                           0.61     19083
   macro avg       0.52      0.58      0.44     19083
weighted avg       0.91      0.61      0.72     19083



In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

n_components = 15  # Set the desired number of components
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"X_train_pca shape: {X_train_pca.shape}")
print(f"X_test_pca shape: {X_test_pca.shape}")

X_train_pca shape: (144972, 15)
X_test_pca shape: (19083, 15)


In [64]:
rf_pca = RandomForestClassifier(random_state=42)
rf_pca.fit(X_train_pca, y_train)

# Predict the labels for the test data using the PCA-transformed features
y_pred_pca = rf_pca.predict(X_test_pca)

# Evaluate the model
print(classification_report(y_test, y_pred_pca))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     18083
           1       0.06      0.01      0.01      1000

    accuracy                           0.94     19083
   macro avg       0.50      0.50      0.49     19083
weighted avg       0.90      0.94      0.92     19083



In [75]:
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(num)

In [79]:
num_sc = pd.DataFrame(numerical_scaled, columns = num.columns)

In [80]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

In [85]:
sel = sel.fit(num_sc)
temp = sel.transform(num_sc)
temp = pd.DataFrame(temp)

In [86]:
print(num_sc.shape)
print(temp.shape)

(181138, 330)
(181138, 90)


In [87]:
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,0.628866,1.000000,0.222222,0.323232,0.343434,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.424242,0.909091,0.909091,0.040404,0.020202,0.010101,0.073333,0.078833,0.153846,0.076923,0.747475,0.262626,0.141414,0.757576,0.686869,0.525253,0.000000,0.000000,0.020202,0.111111,0.383838,0.000000,0.666667,0.000000,0.050505,0.010101,0.000000,0.020202,0.121212,0.535354,0.000000,0.268817,0.242424,0.353535,0.474747,0.343434,0.666667,0.242424,0.626263,0.686869,0.666667,0.696970,0.656566,0.666667,1.000000,0.323232,0.343434,0.888889,0.989899,0.696970,0.980769,0.030303,0.070707,0.232323,0.343434,0.666667,0.595960,0.080808,0.454545,0.545455,0.313131,0.428571,0.4,0.233333,0.500735,1.0,0.000000,0.901639,0.846154,1.0,0.333333,0.642857,0.371134,0.000000,0.818182,0.272727,0.0,0.272727,0.090909
1,0.624862,1.000000,1.000000,0.161616,0.343434,0.888889,1.000000,0.000000,0.000000,0.969697,0.000000,0.212121,0.606061,0.525253,0.393939,0.333333,0.262626,0.769167,0.760333,0.461538,0.538462,0.535354,0.474747,0.070707,0.494949,0.404040,0.666667,0.969697,0.989899,1.000000,1.000000,1.000000,0.858586,0.323232,0.131313,0.000000,0.212121,0.828283,0.898990,0.979798,0.979798,0.478632,0.019969,0.151515,0.242424,0.555556,0.505051,0.505051,0.252525,0.727273,0.797980,0.747475,0.727273,0.626263,0.585859,0.000000,0.161616,0.343434,0.484848,0.919192,0.545455,0.826923,0.000000,0.030303,0.161616,0.181818,0.828283,0.848485,0.131313,1.000000,0.000000,1.000000,0.809524,0.4,0.400000,0.766131,0.0,0.000000,0.426230,0.038462,0.5,0.000000,0.571429,0.000000,0.090909,0.272727,0.181818,0.0,0.454545,0.272727
2,0.793814,1.000000,0.555556,0.272727,0.414141,0.555556,0.000000,0.020202,0.989899,0.939394,0.040404,0.464646,0.777778,0.767677,0.010101,0.000000,0.000000,0.139667,0.156000,0.230769,0.153846,0.838384,0.171717,0.343434,0.808081,0.717172,0.585859,0.040404,0.101010,0.353535,0.585859,0.848485,0.010101,0.535354,0.000000,0.222222,0.000000,0.111111,0.131313,0.373737,0.676768,0.000000,0.308756,0.212121,0.303030,0.525253,0.313131,0.696970,0.121212,0.373737,0.767677,0.767677,0.747475,0.737374,1.000000,0.000000,0.272727,0.414141,0.888889,0.969697,0.828283,0.288462,0.090909,0.393939,0.767677,0.898990,0.111111,0.010101,0.535354,0.191919,0.818182,0.191919,0.333333,0.6,0.383333,0.648635,0.0,0.000000,0.590164,0.750000,1.0,0.333333,0.500000,0.195876,0.545455,0.909091,0.818182,0.0,0.818182,0.090909
3,0.783505,0.500000,0.111111,0.313131,0.353535,0.111111,0.000000,0.000000,1.000000,0.888889,0.000000,0.404040,0.575758,0.555556,0.020202,0.020202,0.010101,0.119667,0.127833,0.230769,0.230769,0.787879,0.222222,0.050505,0.777778,0.646465,0.515152,0.010101,0.040404,0.151515,0.444444,0.838384,0.000000,0.343434,0.000000,0.393939,0.010101,0.060606,0.202020,0.434343,0.787879,0.000000,0.364055,0.363636,0.373737,0.454545,0.474747,0.535354,0.131313,0.686869,0.666667,0.575758,0.626263,0.404040,0.595960,1.000000,0.313131,0.353535,0.363636,0.979798,0.676768,0.384615,0.020202,0.070707,0.585859,0.858586,0.151515,0.000000,0.434343,0.000000,1.000000,0.000000,0.238095,0.4,0.383333,0.925628,1.0,0.000000,0.950820,0.865385,1.0,0.333333,0.500000,0.216495,0.090909,0.090909,1.000000,0.0,1.000000,0.090909
4,0.762887,0.166667,0.777778,0.323232,0.232323,0.000000,0.000000,0.000000,1.000000,0.898990,0.111111,0.505051,0.686869,0.686869,0.010101,0.010101,0.000000,0.182500,0.196667,0.230769,0.153846,0.878788,0.131313,0.060606,0.818182,0.686869,0.555556,0.070707,0.282828,0.565657,0.757576,0.898990,0.010101,0.535354,0.000000,0.292929,0.000000,0.131313,0.151515,0.333333,0.595960,0.051282,0.327189,0.181818,0.222222,0.414141,0.454545,0.555556,0.242424,0.656566,0.868687,0.858586,0.797980,0.666667,0.848485,1.000000,0.323232,0.232323,0.747475,0.989899,0.868687,0.269231,0.242424,0.

In [88]:
sel.variances_

array([2.16751100e-04, 2.12614872e-02, 7.88487886e-02, 8.86348227e-02,
       1.51410570e-03, 2.32932622e-03, 1.35789271e-02, 2.33968564e-02,
       3.29015328e-02, 1.91812618e-03, 2.67900465e-03, 2.14039353e-03,
       1.00670584e-01, 3.16002667e-03, 3.54136481e-03, 3.46724660e-03,
       2.28238410e-01, 9.91548848e-02, 1.62022572e-01, 3.00446640e-03,
       3.22074831e-03, 4.09622470e-02, 2.49202401e-02, 9.98841341e-04,
       5.05288598e-03, 1.87320479e-02, 9.01877577e-04, 1.02554016e-03,
       6.41649987e-04, 1.04512429e-03, 5.63713337e-04, 4.17652155e-04,
       2.85285861e-04, 1.31160559e-02, 4.92729064e-04, 1.64134667e-03,
       1.48975594e-03, 9.95881426e-03, 9.65660780e-03, 9.29586780e-03,
       7.48462319e-03, 6.75158703e-03, 6.57513940e-03, 9.98641496e-03,
       6.82339728e-03, 4.04512612e-03, 3.29657722e-03, 3.63343754e-03,
       6.21247845e-03, 4.02171115e-03, 1.89777401e-03, 1.73344900e-03,
       3.79116581e-03, 4.60637123e-03, 2.67935927e-03, 1.32064612e-03,
      

In [89]:
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())
var_list

[False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True

In [91]:
list(zip(num.columns, var_list))

[('TCODE', False),
 ('AGE', True),
 ('INCOME', True),
 ('WEALTH1', True),
 ('HIT', False),
 ('MALEMILI', False),
 ('MALEVET', False),
 ('VIETVETS', True),
 ('WWIIVETS', True),
 ('LOCALGOV', False),
 ('STATEGOV', False),
 ('FEDGOV', False),
 ('WEALTH2', True),
 ('POP901', False),
 ('POP902', False),
 ('POP903', False),
 ('POP90C1', True),
 ('POP90C2', True),
 ('POP90C3', True),
 ('POP90C4', False),
 ('POP90C5', False),
 ('ETH1', True),
 ('ETH2', True),
 ('ETH3', False),
 ('ETH4', False),
 ('ETH5', False),
 ('ETH6', False),
 ('ETH7', False),
 ('ETH8', False),
 ('ETH9', False),
 ('ETH10', False),
 ('ETH11', False),
 ('ETH12', False),
 ('ETH13', False),
 ('ETH14', False),
 ('ETH15', False),
 ('ETH16', False),
 ('AGE901', False),
 ('AGE902', False),
 ('AGE903', False),
 ('AGE904', False),
 ('AGE905', False),
 ('AGE906', False),
 ('AGE907', False),
 ('CHIL1', False),
 ('CHIL2', False),
 ('CHIL3', False),
 ('AGEC1', False),
 ('AGEC2', False),
 ('AGEC3', False),
 ('AGEC4', False),
 ('AGEC5', F

In [93]:
[col[0] for col in zip(num.columns, var_list) if col[1] == False]  #list of low variance columns: to be dropped 

['TCODE',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C4',
 'POP90C5',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW3',
 'DW7',
 'DW8',
 'DW9',
 'HU3',
 'HU4',
 'HHD1',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HUR1',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA4',
 'HUPA5',
 'HUPA7',
 'DMA',
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC1

In [94]:
removed_columns = pd.DataFrame(data=(num.columns,sel.variances_,sel.get_support()), index=('column_name','variance','statement')).T
removed_columns
drop_list = list(removed_columns[removed_columns['statement'] == False]['column_name'])

In [95]:
drop_list

['TCODE',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C4',
 'POP90C5',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW3',
 'DW7',
 'DW8',
 'DW9',
 'HU3',
 'HU4',
 'HHD1',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HUR1',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA4',
 'HUPA5',
 'HUPA7',
 'DMA',
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC1

In [99]:
num_new = num.drop(drop_list,axis=1)

In [100]:
num_new.shape

(181138, 90)

In [101]:
data = pd.concat((num_new,cat,target),axis=1)
data

,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,ETH2,HHN3,DW1,DW2,DW4,DW5,DW6,HV1,HV2,HV3,HV4,HU1,HU2,HU5,HHD2,HHD3,ETHC2,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR2,HUPA2,HUPA3,HUPA6,RP1,RP2,RP3,RP4,MSA,ADI,IC6,HHAS1,HHAS3,MC1,MC2,PEC2,TPE13,LFC2,LFC4,LFC6,LFC7,LFC8,LFC9,VC1,VC3,POBC2,LSC1,VOC2,HC2,HC4,HC5,HC6,HC7,HC8,HC11,HC13,HC17,HC18,HC19,MHUC1,MHUC2,CARDPROM,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,DOB_YR,DOB_MM,MINRDATE_MM,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_MM,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,TARGET_B,TARGET_D
59366,62.000000,7,2,32,34,9,0,0,99,99,0,42,90,90,4,2,1,440,473,2,1,74,26,14,75,68,52,0,0,2,11,38,0,66,0,5,1,0,2,12,53,0.0,175.0,24,35,47,34,66,24,62,68,66,69,65,66,99,32,34,88,98,69,51,3,7,23,34,66,59,8,45,54,31,9,2,15,96031,1,1,56,45,3,2,92,36,1,10,4,95,4,2.0,IL,H,M,L,F,D,R,1,5.0
87425,61.611649,7,9,16,34,8,99,0,0,96,0,21,60,52,39,33,26,4615,4562,6,7,53,47,7,49,40,66,96,98,99,99,99,85,32,13,0,21,82,89,97,97,4480.0,13.0,15,24,55,50,50,25,72,79,74,72,62,58,0,16,34,48,91,54,43,0,3,16,18,82,84,13,99,0,99,17,2,25,146928,0,1,27,3,2,1,91,0,2,4,3,95,6,4.0,CA,H,other,L,F,A,U,1,10.0
41480,78.000000,7,5,27,41,5,0,2,98,93,4,46,77,76,1,0,0,838,936,3,2,83,17,34,80,71,58,4,10,35,58,84,1,53,0,22,0,11,13,37,67,0.0,201.0,21,30,52,31,69,12,37,76,76,74,73,99,0,27,41,88,96,82,15,9,39,76,89,11,1,53,19,81,19,7,3,24,124395,0,1,37,40,3,2,90,19,7,11,10,95,10,2.0,TX,U,M,L,F,D,T,1,20.0
80970,77.000000,4,1,31,35,1,0,0,99,88,0,40,57,55,2,2,1,718,767,3,3,78,22,5,77,64,51,1,4,15,44,83,0,34,0,39,1,6,20,43,78,0.0,237.0,36,37,45,47,53,13,68,66,57,62,40,59,99,31,35,36,97,67,20,2,7,58,85,15,0,43,0,99,0,5,2,24,177516,1,1,59,46,3,2,90,21,2,2,12,95,12,2.0,other,H,M,L,F,C,R,1,37.0
94582,75.000000,2,7,32,23,0,0,0,99,89,11,50,68,68,1,1,0,1095,1180,3,2,87,13,6,81,68,55,7,28,56,75,89,1,53,0,29,0,13,15,33,59,480.0,213.0,18,22,41,45,55,24,65,86,85,79,66,84,99,32,23,74,98,86,14,24,47,80,87,13,0,51,0,99,0,6,2,31,21885,1,1,16,40,3,2,86,22,8,6,11,95,4,7.0,NC,H,M,L,G,C,T,1,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95406,50.000000,6,6,26,46,6,0,0,99,88,8,46,59,58,1,0,0,733,875,4,3,87,13,25,84,74,56,6,10,23,47,87,0,47,0,40,0,13,36,67,75,5960.0,329.0,23,31,37,44,56,48,69,78,75,72,55,99,99,37,41,41,95,72,13,18,44,83,92,8,0,69,0,99,0,6,1,23,35088,0,4,43,16,3,2,90,48,3,2,3,96,2,3.0,FL,H,F,L,D,C,S,0,0.0
95407,61.611649,5,9,47,11,9,99,0,0,78,10,48,54,38,32,24,14,988,1025,6,6,56,44,3,72,56,55,2,10,49,73,92,0,40,8,13,12,69,84,92,97,380.0,0.0,11,9,70,63,37,2,69,89,83,69,57,61,94,48,11,26,92,60,14,7,32,78,91,9,86,12,93,7,98,16,4,6,184568,0,1,12,27,3,2,96,0,2,2,2,96,2,2.0,other,H,M,L,G,C,C,0,0.0
95408,48.000000,7,9,43,19,9,96,0,4,91,3,70,89,88,1,0,0,1679,1723,3,3,88,12,0,89,85,58,18,71,88,91,97,5,77,0,10,0,16,26,44,79,3360.0,201.0,8,8,61,83,17,3,61,88,87,48,43,99,0,43,19,51,94,84,7,55,90,94,94,6,82,16,69,31,67,18,5,4,122706,1,1,2,24,3,1,96,50,1,3,3,96,3,3.0,TX,H,M,L,F,A,C,0,0.0
95409,60.000000,5,9,46,20,9,99,0,0,82,14,44,96,95,2,1,0,376,377,4,3,66,34,10,64,43,49,0,0,0,1,9,0,45,0,0,0,3,34,78,91,4040.0,61.0,21,10,26,43,57,25,69,70,70,77,24,62,0,46,20,82,92,42,51,0,0,0,0,99,97,0,99,0,99,5,2,14,189641,1,3,34,30,3,3,95,38,1,3,1,96,10,10.0,MI,H,M,L,E,B,C,0,0.0
